In [1]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install kss
!pip install okt
!pip install konlpy
!pip install gluonnlp
!pip install mxnet
!pip install sentence-transformers


  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-nh_z4lwk
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-nh_z4lwk
  Using cached okt-0.0.3-py3-none-any.whl (4.5 kB)
ERROR: Could not find a version that satisfies the requirement oktapysdk (from okt) (from versions: none)
ERROR: No matching distribution found for oktapysdk


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import sys

sys.path.append('/content/drive/MyDrive/ML_Train/Backend_ML')

# from model import Model
from sentiment.sentiment_model import get_sentiment_model_class
from keyword_extractor.keyword_model import get_keyword_model_class

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [5]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
from sklearn.model_selection import train_test_split
                                          

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
import torch
import torch.nn
import itertools
import random
import numpy as np
from tqdm.notebook import tqdm
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
import os
import yaml
import gluonnlp as nlp
from konlpy.tag import Okt
from kss import split_sentences
import gluonnlp as nlp

cfg = {}

with open('/content/drive/MyDrive/ML_Train/Backend_ML/sentiment/configs/cfg.yaml') as f:
    sentiment_cfg = yaml.load(f, Loader=yaml.FullLoader)

with open('/content/drive/MyDrive/ML_Train/Backend_ML/keyword_extractor/configs/cfg.yaml') as f:
    keyword_cfg = yaml.load(f, Loader=yaml.FullLoader)

# mm = Model(cfg)

# x = '솔직히 말해 그다지 예쁜 여자아이는 아니다.눈에 띄는 데가 있는 것도 아니다.멋진 옷을 입고 있는 것도 아니다.머리카락 뒤쪽에는 나쁜 잠버릇이 끈질기게 달라붙어 있고,나이도 적지 않다.벌써 서른 살에 가까울테니까.엄밀히 말하면 여자아이라고 할 수도 없으리라.그럼에도 불구하고, 나는 50미터 떨어진 곳에서부터그녀를 알아볼 정도다.그녀는 내게 있어서 100%의 여자아이이기 때문이다.그녀의 모습을 목격하는 순간부터내 가슴은 땅울림처럼 떨리고,입안은 사막처럼 바싹 말라버린다.'



sentiment_model_class = get_sentiment_model_class(sentiment_cfg['model_name'])
sentiment_model = sentiment_model_class(sentiment_cfg)

keyword_model_class = get_keyword_model_class(keyword_cfg['model_name'])
keyword_model = keyword_model_class(keyword_cfg)


    
    

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [17]:
x = '오늘은 내 생일이다. 거기다가 야근도 없어서 일찍 퇴근했다. 생일이라고 친구들이 술을 사준다고 해서 기대했다. 기대한대로 평소에 자주 못 먹던 비싼 음식들을 사줬다.'
print(keyword_model.predict(x))
print(sentiment_model.predict(x))
print('\n\n')



4 11
['음식', '친구', '생일']
문장 별 score
문장 : 오늘은 내 생일이다.
{'공포': 0.009190887212753296, '놀람': 0.009943624027073383, '분노': 0.009912450797855854, '슬픔': 0.01710125245153904, '중립': 0.007281970232725143, '행복': 0.9360285997390747, '혐오': 0.010541143827140331}
행복

문장 : 거기다가 야근도 없어서 일찍 퇴근했다.
{'공포': 0.009418625384569168, '놀람': 0.018772009760141373, '분노': 0.013864248991012573, '슬픔': 0.8362667560577393, '중립': 0.013999137096107006, '행복': 0.0999278873205185, '혐오': 0.00775131955742836}
슬픔

문장 : 생일이라고 친구들이 술을 사준다고 해서 기대했다.
{'공포': 0.1660534292459488, '놀람': 0.044254597276449203, '분노': 0.0109171187505126, '슬픔': 0.02919216826558113, '중립': 0.007913611829280853, '행복': 0.7324648499488831, '혐오': 0.009204299189150333}
행복

문장 : 기대한대로 평소에 자주 못 먹던 비싼 음식들을 사줬다.
{'공포': 0.009935864247381687, '놀람': 0.025145279243588448, '분노': 0.016310658305883408, '슬픔': 0.0992869883775711, '중립': 0.0127415731549263, '행복': 0.8297435641288757, '혐오': 0.006835957523435354}
행복

총 score
{'공포': 0.03035816363990307, '놀람': 0.03333830460906029, '분노': 

In [18]:
x += ' 하지만 기대했던 맛이 아니여서 별로였다. 게다가 집에 와보니, 지갑도 어디있는지 안 보인다. 최악의 하루였다.'

print(keyword_model.predict(x))
print(sentiment_model.predict(x))

6 17
['음식', '친구', '생일']
문장 별 score
문장 : 오늘은 내 생일이다.
{'공포': 0.009190882556140423, '놀람': 0.009943624027073383, '분노': 0.009912455454468727, '슬픔': 0.01710125245153904, '중립': 0.007281970232725143, '행복': 0.9360285997390747, '혐오': 0.010541143827140331}
행복

문장 : 거기다가 야근도 없어서 일찍 퇴근했다.
{'공포': 0.009418625384569168, '놀람': 0.01877201534807682, '분노': 0.013864248991012573, '슬픔': 0.8362667560577393, '중립': 0.013999137096107006, '행복': 0.0999278873205185, '혐오': 0.00775131955742836}
슬픔

문장 : 생일이라고 친구들이 술을 사준다고 해서 기대했다.
{'공포': 0.16605345904827118, '놀람': 0.044254593551158905, '분노': 0.010917117819190025, '슬픔': 0.029192166402935982, '중립': 0.007913610897958279, '행복': 0.7324647903442383, '혐오': 0.009204298257827759}
행복

문장 : 기대한대로 평소에 자주 못 먹던 비싼 음식들을 사줬다. 하지만 기대했던 맛이 아니여서 별로였다.
{'공포': 0.009447154588997364, '놀람': 0.004978714045137167, '분노': 0.00860879011452198, '슬픔': 0.9319069385528564, '중립': 0.007065629120916128, '행복': 0.026874326169490814, '혐오': 0.011118393391370773}
슬픔

문장 : 게다가 집에 와보니, 지갑도 어디있는지 안 보인다.
{'공포':

In [ ]:
keyword_cfg['model_name'] = 'KoBERT'
keyword_cfg['model_path'] = '/content/drive/MyDrive/model_weight/KoBERT/53_18_7class.pth'

keyword_model_class2 = get_keyword_model_class(keyword_cfg['model_name'])
keyword_model2 = keyword_model_class(keyword_cfg)


In [20]:
x = '오늘은 내 생일이다. 거기다가 야근도 없어서 일찍 퇴근했다. 생일이라고 친구들이 술을 사준다고 해서 기대했다. 기대한대로 평소에 자주 못 먹던 비싼 음식들을 사줬다.'
print(keyword_model2.predict(x))

x += '하지만 기대했던 맛이 아니여서 별로였다. 게다가 집에 와보니, 지갑도 어디있는지 안 보인다. 최악의 하루였다.'

print(keyword_model2.predict(x))


4 11
['음식', '친구', '생일']
6 17
['음식', '친구', '생일']
